In [1]:
import csv
import numpy as np
import pandas as pd
import time
from urllib2 import urlopen # python2
import urllib2

In [2]:
raw_offisde = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3003377s-offsides.tsv")
stitch_cid = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv")

In [3]:
print len(raw_offisde)
raw_offisde.head()

438801


,stitch_id,drug,umls_id,event,rr,log2rr,t_statistic,pvalue,observed,expected,bg_correction,sider,future_aers,medeffect
0,CID000000076,dehydroepiandrosterone,C0000737,abdominal pain,2.250000,1.169925,6.537095,6.156712e-07,9,4.000000,0.002849,0,0,0
1,CID000000076,dehydroepiandrosterone,C0001622,hyperadrenalism,11.000000,3.459432,4.782699,1.644408e-03,2,0.181818,0.000041,0,0,0
2,CID000000076,dehydroepiandrosterone,C0001623,adrenal insufficiency,2.200000,1.137504,4.315199,9.884952e-03,2,0.909091,0.002491,0,0,0
3,CID000000076,dehydroepiandrosterone,C0002792,anaphylactic reaction,2.588235,1.371969,4.590918,3.355380e-03,4,1.545455,0.000503,0,0,0
4,CID000000076,dehydroepiandrosterone,C0002940,aneurysm,7.333333,2.874469,4.598374,3.399457e-03,2,0.272727,0.000135,0,0,0


In [4]:
print(len(raw_offisde))
offside = raw_offisde[['stitch_id','drug', 'umls_id', 'event']]
offside.head()

438801


,stitch_id,drug,umls_id,event
0,CID000000076,dehydroepiandrosterone,C0000737,abdominal pain
1,CID000000076,dehydroepiandrosterone,C0001622,hyperadrenalism
2,CID000000076,dehydroepiandrosterone,C0001623,adrenal insufficiency
3,CID000000076,dehydroepiandrosterone,C0002792,anaphylactic reaction
4,CID000000076,dehydroepiandrosterone,C0002940,aneurysm


In [5]:
offside_wo_dup = offside.drop_duplicates(subset=['stitch_id', 'drug'])
print(len(offside_wo_dup))
offside_wo_dup.head()

1332


,stitch_id,drug,umls_id,event
0,CID000000076,dehydroepiandrosterone,C0000737,abdominal pain
179,CID000000085,carnitine,C0002871,anaemia
452,CID000000119,GABA,C0000833,abscess
940,CID000000143,leucovorin,C0000737,abdominal pain
1591,CID000000158,PGE2,C0000832,premature separation of placenta


In [57]:
print len(offside.drop_duplicates(subset='umls_id'))
print len(offside.drop_duplicates(subset='event'))

10097
10097


In [6]:
stitch_cid.head(1)

,flat_chemical,stereo_chemical,source_name,source_id
0,CID100000001,CID000000001,PC,1


In [12]:
# using UniChem REST API, convert pubchemID to DrugBank ID
with open("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/4_OFFSIDE_DrugBank_id.tsv", "w") as resultFile:
    features = ['STITCH_compound_id_stereo', 'PubChem_ID', 'DrugBank_ID']
    resultTsv = csv.DictWriter(resultFile, features, dialect=csv.excel_tab)
    resultTsv.writeheader()
    
    cnt = 0
    result = list()
    url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'
    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
    for index, row in offside_wo_dup[cnt:].iterrows():
        record = row.tolist()
        found = stitch_cid[stitch_cid.stereo_chemical == record[0]]
        pubchem_ids = found['source_id']
        for j in pubchem_ids:
            url = url_base + str(j) + '/22/2'
            try:
                f=urlopen(url)
            except urllib2.HTTPError, e:
                if e.code == 404:
                    cnt = cnt + 1
                    if cnt % 1000 == 0 :
                        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
                    continue
                else:
                    print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                    raise
            except urllib2.URLError, e:
                if e.args[0][0] == 110: # connection time out
                    sleep_secs = attempt ** 2
                    print e
                    print url
                    print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    print '    Retrying in %d seconds...' % sleep_secs
                    time.sleep(sleep_secs)
                    cnt= cnt +1
                    if cnt % 1000 == 0 :
                        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
                    continue
                else:
                    print e
                    print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                    raise
        
            else:
                all_data = f.readlines()
                if all_data[0] == '[]':
                    cnt = cnt + 1
                else:
                    db_id=all_data[0]
                    #result.append([record[0], record[1], db_id])
                    resultTsv.writerow({'STITCH_compound_id_stereo':record[0],'PubChem_ID' : j, 'DrugBank_ID' : db_id})
                    cnt = cnt + 1

                if cnt % 1000 == 0:
                    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

0 : Sun 20:37:35
100 : Sun 20:39:34
200 : Sun 20:41:28
300 : Sun 20:43:17
400 : Sun 20:45:14
500 : Sun 20:47:18
600 : Sun 20:49:11
700 : Sun 20:51:04
800 : Sun 20:52:59
900 : Sun 20:54:57
1000 : Sun 20:56:48
1100 : Sun 20:58:55
1200 : Sun 21:00:51
1300 : Sun 21:02:47
1400 : Sun 21:04:42
1500 : Sun 21:06:42
1600 : Sun 21:08:42
1700 : Sun 21:10:54
1800 : Sun 21:13:03
1900 : Sun 21:15:14
2000 : Sun 21:17:21
2100 : Sun 21:19:21
2200 : Sun 21:21:25
2300 : Sun 21:23:23
2400 : Sun 21:25:33
2500 : Sun 21:27:44
2600 : Sun 21:29:45
2700 : Sun 21:31:52
2800 : Sun 21:33:52
2900 : Sun 21:35:57
3000 : Sun 21:38:12
3100 : Sun 21:40:14
3200 : Sun 21:42:24
3300 : Sun 21:44:26
3400 : Sun 21:46:28
3500 : Sun 21:48:37
3600 : Sun 21:50:32
3700 : Sun 21:52:34
3800 : Sun 21:54:46
3900 : Sun 21:56:57
4000 : Sun 21:59:02
4100 : Sun 22:01:09
4200 : Sun 22:03:06
4300 : Sun 22:05:03
4400 : Sun 22:07:06
4500 : Sun 22:09:18
4600 : Sun 22:11:16
4700 : Sun 22:13:25
4800 : Sun 22:15:21
4900 : Sun 22:17:28
5000 : Sun 2

In [13]:
cnt

8878

In [14]:
raw_db = pd.read_table('/home/share/aschoi/data/Drug_Repositioning/5_new_drug/4_1_raw_OFFSIDE_DrugBank_id.tsv')

In [73]:
raw_db.head()

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID000000119,119,"[{""src_compound_id"":""DB02530""}]"
1,CID000000119,6992099,"[{""src_compound_id"":""DB02530""}]"
2,CID000000338,338,"[{""src_compound_id"":""DB00936""}]"
3,CID000000444,444,"[{""src_compound_id"":""DB01156""}]"
4,CID000000564,517030,"[{""src_compound_id"":""DB00513""}]"


In [16]:
stitch_cid[stitch_cid.stereo_chemical == 'CID000000119']

,flat_chemical,stereo_chemical,source_name,source_id
2067610,CID100000119,CID000000119,PC,119
7773276,CID100000119,CID000000119,PC,210329
8447122,CID100000119,CID000000119,PC,21864380
8515502,CID100000119,CID000000119,PC,21947455
9915952,CID100000119,CID000000119,PC,23663645
9933817,CID100000119,CID000000119,PC,23694842
19188034,CID100000119,CID000000119,PC,449579
25499710,CID100000119,CID000000119,PC,53470905
26456946,CID100000119,CID000000119,PC,5460232
32385341,CID100000119,CID000000119,PC,60198381


In [17]:
offside_wo_dup[offside_wo_dup.stitch_id == 'CID000000119']

,stitch_id,drug,umls_id,event
452,CID000000119,GABA,C0000833,abscess


In [50]:
print len(raw_db)
print len(raw_db.drop_duplicates(subset = 'STITCH_compound_id_stereo'))

586
554


In [21]:
se_db_id = raw_db['DrugBank_ID']

In [44]:
raw_db.head(1)

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID000000119,119,"[{""src_compound_id"":""DB02530""}]"


In [47]:
result = pd.DataFrame()
for index, row in raw_db.iterrows():
    record = row.tolist()
    db_ids = record[2].split(',')
    for i in db_ids:
        db_id = i.split(':')[1].lstrip('\"').rstrip('\"}]')
        temp_df = pd.DataFrame([[record[0], record[1], db_id]], columns = ['STITCH_compound_id_stereo', 'PubChem_ID', 'DrugBank_ID'])
        result = result.append(temp_df)
result.index = range(0, len(result))
    
        

In [59]:
print len(result)
print len(result.drop_duplicates(subset = ['STITCH_compound_id_stereo', 'DrugBank_ID']))
print len(result.drop_duplicates(subset = ['STITCH_compound_id_stereo']))
print len(result.drop_duplicates(subset = ['DrugBank_ID']))

590
558
554
558


In [61]:
result.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/4_2_OFFSIDE_DrugBank_id.tsv", sep='\t', index=False)

In [74]:
result.head()

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID000000119,119,DB02530
1,CID000000119,6992099,DB02530
2,CID000000338,338,DB00936
3,CID000000444,444,DB01156
4,CID000000564,517030,DB00513


In [75]:
result[result.duplicated(subset = ['STITCH_compound_id_stereo', 'DrugBank_ID'], keep=False)].sort_values(by = 'DrugBank_ID')
#result[result.duplicated(subset = ['DrugBank_ID'], keep=False)].sort_values(by = 'DrugBank_ID')

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
476,CID000041781,51371498,DB00214
475,CID000041781,41781,DB00214
433,CID000006116,6116,DB00258
432,CID000006116,517040,DB00258
4,CID000000564,517030,DB00513
5,CID000000564,564,DB00513
115,CID000002764,4011971,DB00537
113,CID000002764,2764,DB00537
559,CID000444013,6435809,DB00538
558,CID000444013,444013,DB00538


In [67]:
stitch_db_table = result[['STITCH_compound_id_stereo', 'DrugBank_ID']].drop_duplicates()

In [69]:
len(stitch_db_table)

558

In [68]:
stitch_db_table.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/4_3_OFFSIDE_STITCHid_DrugBankid.tsv", sep='\t', index=False)

In [72]:
stitch_db_table[stitch_db_table.duplicated(subset='STITCH_compound_id_stereo', keep=False)]

,STITCH_compound_id_stereo,DrugBank_ID
11,CID000000753,DB04077
12,CID000000753,DB09462
113,CID000002764,DB00537
114,CID000002764,DB05488
410,CID000005578,DB03125
412,CID000005578,DB00440
550,CID000193962,DB06414
551,CID000193962,DB07191


In [78]:
result[result.STITCH_compound_id_stereo =='CID000000753']

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
11,CID000000753,753,DB04077
12,CID000000753,753,DB09462


In [79]:
result[result.STITCH_compound_id_stereo =='CID000002764']

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
113,CID000002764,2764,DB00537
114,CID000002764,2764,DB05488
115,CID000002764,4011971,DB00537
116,CID000002764,4011971,DB05488


In [80]:
result[result.STITCH_compound_id_stereo =='CID000005578']

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
410,CID000005578,3382784,DB03125
411,CID000005578,446998,DB03125
412,CID000005578,5578,DB00440


In [81]:
result[result.STITCH_compound_id_stereo =='CID000193962']

,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
550,CID000193962,193962,DB06414
551,CID000193962,193962,DB07191
